In [ ]:
import pandas as pd

df = pd.read_excel('/content/sample_data/exp_data.xlsx')

data = df.to_dict('records')

In [ ]:
data[:3]

[{'Item': 'Критики хвалили картину художницы, часто выставлявшейся в местных галереях.\nЧасто выставлялась… а) картина б) художница',
  'ex_item': 'ПЗ',
  'animacy': 'НО',
  '1st_noun': 'а) картина'},
 {'Item': 'Конструктор переживал из–за двигателя автомобиля, внезапно сломавшегося после гонки.\nСломался после гонки… а) автомобиль б) двигатель',
  'ex_item': 'НЗ',
  'animacy': 'НН',
  '1st_noun': 'б) двигатель'},
 {'Item': 'Акционеры ругали политику компании, совсем не учитывавшую их интересы.\nНе учитывала интересы… а) компания б) политика',
  'ex_item': 'РЗ',
  'animacy': 'НН',
  '1st_noun': 'б) политика'}]

In [ ]:
# no cash
import os
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import random
key = userdata.get('OPENAI_KEY')
client = OpenAI(api_key=key)

res = []

for i in range(60):
  shuffled_data = random.sample(data, len(data))

  for item in shuffled_data:
    chat_completion = client.chat.completions.create(
              messages=[
                        {
                                        "role": "user",
                                        "content": f"Прочитайте предложение и ответьте на вопрос по его содержанию. Пожалуйста, \
                                        ответьте только одним словом: выбранным вариантов ответа.\n{item['Item']}.",
                                        }
                            ],
                  model="gpt-4o",
                  temperature=0,
              )
    response = chat_completion.choices[0].message.content.lower()
    #print(response)
    if item['ex_item'] == 'РЗ':
      if response.lower() in item['1st_noun'] or item['1st_noun'] in response.lower():
        ans = '1'
      else:
        ans = '0'
    elif item['ex_item'] == 'ПЗ':
      if response.lower() in item['1st_noun'] or item['1st_noun'] in response.lower():
        ans = '0'
      else:
        ans = '1'
    elif item['ex_item'] == 'НЗ':
      if response.lower() in item['1st_noun'] or item['1st_noun'] in response.lower():
        ans = 'РЗ'
      else:
        ans = 'ПЗ'
    else:
      ans = '-'
    res.append({"model": "gpt-4o", "item": item['Item'], "response": response,
                "ex_item": item['ex_item'], "animacy": item['animacy'], "right_answer": ans})

res_df = pd.DataFrame(res)
res_df.to_excel('/content/sample_data/result_.xlsx', index=False, sheet_name='gpt')

In [ ]:
from google.colab import files
files.download("/content/sample_data/result_.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>